In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
bbn_path = base_dir + "BBN/BBN/"

Mounted at /content/drive/


In [2]:
import json

with open(bbn_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [3]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  32739 elementi


In [4]:
# print(len(examples_train[14535]['mentions']))
# print(len(examples_train[14535]['tokens']))
# prod = 1
# sum=0
# for elem in examples_train[14535]['mentions']:
#    prod *= len(elem)
#    sum+=len(elem)

# print(prod)
# print(sum/len(examples_train[14535]['mentions']))

In [4]:
# rimuovo elemento 14535, caso problematico in cui generare tutte le permutazioni della frase avrebbe generato 7625597484987 frasi.
# questo elemento è una frase con 77 tokens, di cui 27 menzioni con ciascuna in media 3 tipi
# del examples_train[14535]
examples_iter = iter(examples_train)

In [6]:
#examples_iter = iter(examples_train[len(examples_train)//2:len(examples_train)+1])

In [5]:
def get_permutations(i, threshold):

  d = sorted(i, key=operator.itemgetter("start"))
  d = {k: list(v) for k, v in itertools.groupby(d, key=operator.itemgetter("start"))}

  all_list=[]
  for key, value in d.items():
    all_list.append(value)

  p = list(itertools.product(*all_list))

  sum=0
  for per in p:
    prod=1
    for i in per:
      prod*= len(i)
    sum+=prod

  if (sum>threshold):
    return [], [], sum
  else:

    total_perm=[]
    count_perm=0
    for per in p:
      mini_list = []
      for lista_labels in per:
        mini_list.append(lista_labels['labels'])

      p2 = list(itertools.product(*mini_list))
      total_perm.append(p2)

    return total_perm.copy(), p.copy(), sum






In [6]:
import operator
import itertools
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords
import json

perm_a_priori=[]
perm_effettive=[]
sottostime=[]
lunghezze={
    'minori=_10':0,
    'maggiori_10_minori=20':0,
    'maggiori_20_minori=50':0,
    'maggiori_50_minori=100':0,
    'maggiori_100_minori=200':0,
    'maggiori_200':0,
    'sottostimate': 0,
    'sovrastimate': 0,
    'frasi_con_overlap':0,
}

fp = open(path_corpus_data + 'corpus_data_entire_bbn.txt', "w")
restricted_set = set()

for num,i in enumerate(tqdm(examples_iter)):
  dizionario_permutations = {'pos':num, 'num': 0,'istanza':i}
  istance = i['mentions'][:]
  if len(istance)>1:
    
    permutations_2, permutations, num_permutations = get_permutations(istance, 20)
    
    dizionario_permutations['num'] = num_permutations
    

    iteri=0

    for p_1 in permutations:
      for p_2 in iter(permutations_2[iteri]):
        
        phrase = i['tokens'][:]
        num_pop=0
        for h in range(0, len(p_2)):
          start = permutations[iteri][h]['start']
          end = permutations[iteri][h]['end']

          phrase[start-num_pop:end-num_pop] = [p_2[h]]
          if((end-start)>1):
            num_pop += (end - start) - 1

          restricted_set.add(p_2[h])

        
        fp.write((remove_stopwords(' '.join(phrase)) + '\n'))

      iteri+=1
    
  else:
    dizionario_permutations['num'] = len(istance[0]['labels'])
    for k in istance[0]['labels']:
      phrase = i['tokens'][:]
      if (i['mentions'][0]['end'] - i['mentions'][0]['start'] ==1):          
        phrase[i['mentions'][0]['start']] = k
        restricted_set.add(k)
      else:
        phrase[i['mentions'][0]['start']:i['mentions'][0]['end']] = [k]
        restricted_set.add(k)

      fp.write((remove_stopwords(' '.join(phrase)) + '\n'))
  
  with open(path_corpus_data + 'data.json', 'a') as fp_1:
      json.dump(dizionario_permutations, fp_1)

fp.close()

32739it [02:00, 272.12it/s]


In [38]:
dizionario = {0:[[1,2,7],[3,4,8]], 1: [[5,6]], 2:[[1,2],[3,4]] }

num_permutations = 0
#for key,value in dizionario.items():
  #if len(list(value))>1:
    #print(key ,'->', value)
  #else:
    #print("sono lunga uno")

 #((3*2*2)+(3*2*2)) + ((3*2*2)+(3*2*2)) = 48
all_list=[]
for key, value in dizionario.items():
    all_list.append(value)

permutations = list(itertools.product(*all_list))
print(permutations)
sum=0
# for p in permutations:
#   prod=1
#   for i in p:
#     prod*= len(i)
#   sum+=prod

# print(sum)
total_perm=[]
tot_perm=0
for p in permutations:
    mini_list = []
    for lista_labels in p:
      mini_list.append(lista_labels)

    permutations_2 = list(itertools.product(*mini_list))
    tot_perm+=len(permutations_2)
    total_perm.append(permutations_2)

    print(permutations_2)

print(total_perm)
print(tot_perm)
if total_perm > threshold:
  continue 
else
# inizia a creare le frasi



[([1, 2, 7], [5, 6], [1, 2]), ([1, 2, 7], [5, 6], [3, 4]), ([3, 4, 8], [5, 6], [1, 2]), ([3, 4, 8], [5, 6], [3, 4])]
[(1, 5, 1), (1, 5, 2), (1, 6, 1), (1, 6, 2), (2, 5, 1), (2, 5, 2), (2, 6, 1), (2, 6, 2), (7, 5, 1), (7, 5, 2), (7, 6, 1), (7, 6, 2)]
[(1, 5, 3), (1, 5, 4), (1, 6, 3), (1, 6, 4), (2, 5, 3), (2, 5, 4), (2, 6, 3), (2, 6, 4), (7, 5, 3), (7, 5, 4), (7, 6, 3), (7, 6, 4)]
[(3, 5, 1), (3, 5, 2), (3, 6, 1), (3, 6, 2), (4, 5, 1), (4, 5, 2), (4, 6, 1), (4, 6, 2), (8, 5, 1), (8, 5, 2), (8, 6, 1), (8, 6, 2)]
[(3, 5, 3), (3, 5, 4), (3, 6, 3), (3, 6, 4), (4, 5, 3), (4, 5, 4), (4, 6, 3), (4, 6, 4), (8, 5, 3), (8, 5, 4), (8, 6, 3), (8, 6, 4)]
[[(1, 5, 1), (1, 5, 2), (1, 6, 1), (1, 6, 2), (2, 5, 1), (2, 5, 2), (2, 6, 1), (2, 6, 2), (7, 5, 1), (7, 5, 2), (7, 6, 1), (7, 6, 2)], [(1, 5, 3), (1, 5, 4), (1, 6, 3), (1, 6, 4), (2, 5, 3), (2, 5, 4), (2, 6, 3), (2, 6, 4), (7, 5, 3), (7, 5, 4), (7, 6, 3), (7, 6, 4)], [(3, 5, 1), (3, 5, 2), (3, 6, 1), (3, 6, 2), (4, 5, 1), (4, 5, 2), (4, 6, 1), (4, 

In [ ]:
import statistics
print(f"frasi con overlap in BBN {lunghezze['frasi_con_overlap']}")
print(f"con stima a priori maggiore di effettiva {lunghezze['sovrastimate']}")
print(f"con stima a priori minore di effettiva {lunghezze['sottostimate']}")
print(f"mediana delle stime a priori {statistics.median(perm_a_priori)}")
print(f"mediana delle permutazioni effettive {statistics.median(perm_effettive)}")
print(f"nei {lunghezze['sottostimate']} casi in cui ho sottostimato la media della sottostima è di {sum(sottostime)/len(sottostime)}")

frasi con overlap in BBN 1131
con stima a priori maggiore di effettiva 307
con stima a priori minore di effettiva 824
mediana delle stime a priori 3
mediana delle permutazioni effettive 6
nei 824 casi in cui ho sottostimato la media della sottostima è di 1.6960558252427178


In [ ]:
restricted_list = list(restricted_set)

In [ ]:
import pickle
with open(path_corpus_data + 'restricted_list_bbn.pkl', 'wb') as fp:
  pickle.dump(restricted_list, fp)